# Building the Transformer from Scratch

In this notebook, we'll be implementing the famous Transformer architecture from scratch.

The code is based off of the following repos/blog posts:

- [attention-is-all-you-need-pytorch](https://github.com/jadore801120/attention-is-all-you-need-pytorch)
- [pytorch-pretrained-BERT](https://github.com/huggingface/pytorch-pretrained-BERT)
- [The Annotated Transformer](http://nlp.seas.harvard.edu/2018/04/03/attention.html) 

Thanks so much to their authors!

In [1]:
import torch
import torch.nn as nn
import numpy as np

One of the keys to understanding how any model works is understanding how the shapes of the tensors change during the processing of each part. We'll be using the logging module to output debugging information to help our understanding.

In [2]:
import logging
logger = logging.getLogger("tensor_shapes")
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
# if you want the model to continuously print tensor shapes, set to DEBUG!
logger.setLevel(1)

In [3]:
import inspect
def getclass():
    stack = inspect.stack()
    return stack[3][0].f_locals["self"].__class__

# A helper function to check how tensor sizes change
def log_size(tsr: torch.Tensor, name: str):
    cls = getclass()
    logger.log(level=cls.level, msg=f"[{cls.__name__}] {name} size={tsr.shape}")

We'll use logging levels to control the modules we receive output from. The lower the logging level, the more tensor information you'll get. Feel free to play around!

In [4]:
from enum import IntEnum
# Control how much debugging output we want
class TensorLoggingLevels(IntEnum):
    attention = 1
    attention_head = 2
    multihead_attention_block = 3
    enc_dec_block = 4
    enc_dec = 5

We'll be using an enum to refer to dimensions whenever possible to improve readability.

In [5]:
class Dim(IntEnum):
    batch = 0
    seq = 1
    feature = 2

# Components

### Scaled dot product attention

The Transformer is an attention-based architecture. The attention used in the Transformer is the scaled dot product attention, represented by the following formula.

$$ \textrm{Attention}(Q, K, V) = \textrm{softmax}(\frac{QK^T}{\sqrt{d_k}})V $$

![image](https://i2.wp.com/mlexplained.com/wp-content/uploads/2017/12/scaled_dot_product_attention.png?zoom=2&w=750)

In [6]:
import math

class ScaledDotProductAttention(nn.Module):
    level = TensorLoggingLevels.attention # Logging level: 
    def __init__(self, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(dropout)

    def forward(self, q, k, v, mask=None):
        d_k = k.size(-1) # get the size of the key
        assert q.size(-1) == d_k

        # compute the dot product between queries and keys for
        # each batch and position in the sequence
        attn = torch.bmm(q, k.transpose(Dim.seq, Dim.feature)) # (Batch, Seq, Seq)
        # we get an attention score between each position in the sequence
        # for each batch

        # scale the dot products by the dimensionality (see the paper for why we do this!)
        attn = attn / math.sqrt(d_k)
        # normalize the weights across the sequence dimension
        # (Note that since we transposed, the sequence and feature dimensions are switched)
        attn = torch.exp(attn)
        log_size(attn, "attention weight") # (Batch, Seq, Seq)
        
        # fill attention weights with 0s where padded
        if mask is not None: attn = attn.masked_fill(mask, 0)
        attn = attn / attn.sum(dim=-1, keepdim=True)
        attn = self.dropout(attn)
        output = torch.bmm(attn, v) # (Batch, Seq, Feature)
        log_size(output, "attention output size") # (Batch, Seq, Seq)
        return output

In [7]:
attn = ScaledDotProductAttention()

In [8]:
q = torch.rand(5, 10, 20)
k = torch.rand(5, 10, 20)
v = torch.rand(5, 10, 20)

In [9]:
attn(q, k, v)

[ScaledDotProductAttention] attention weight size=torch.Size([5, 10, 10])
[ScaledDotProductAttention] attention output size size=torch.Size([5, 10, 20])


tensor([[[0.5621, 0.4720, 0.5329, 0.4297, 0.5339, 0.4660, 0.7556, 0.6481,
          0.5159, 0.5285, 0.3473, 0.5580, 0.5791, 0.6454, 0.4410, 0.6190,
          0.5605, 0.6490, 0.5019, 0.4977],
         [0.4839, 0.3685, 0.4248, 0.4280, 0.4808, 0.3795, 0.7199, 0.5827,
          0.4373, 0.4643, 0.2958, 0.3993, 0.4992, 0.6386, 0.4102, 0.4555,
          0.4649, 0.5748, 0.4454, 0.4761],
         [0.5586, 0.4803, 0.5412, 0.4233, 0.5409, 0.4835, 0.7533, 0.6576,
          0.5251, 0.5308, 0.3540, 0.5542, 0.5887, 0.6446, 0.4336, 0.6424,
          0.5745, 0.6643, 0.5125, 0.5048],
         [0.5396, 0.4681, 0.5228, 0.4202, 0.5556, 0.4575, 0.7571, 0.6587,
          0.5126, 0.5396, 0.3577, 0.5636, 0.5809, 0.6459, 0.4377, 0.6174,
          0.5579, 0.6536, 0.4923, 0.4734],
         [0.4879, 0.4757, 0.5197, 0.3979, 0.4973, 0.4596, 0.6580, 0.6172,
          0.4789, 0.5292, 0.3193, 0.5004, 0.5664, 0.5976, 0.3650, 0.6486,
          0.5525, 0.6514, 0.4817, 0.4642],
         [0.5560, 0.4693, 0.5312, 0.4282, 0.5

### Multi-Head Attention

Now, we turn to the core component in the Transformer architecture: the multi-head attention block. This block applies linear transformations to the input, then applies scaled dot product attention.

![image](https://i2.wp.com/mlexplained.com/wp-content/uploads/2017/12/multi_head_attention.png?zoom=2&resize=224%2C293)

In [15]:
class AttentionHead(nn.Module):
    level = TensorLoggingLevels.attention_head
    def __init__(self, d_model, d_feature, dropout=0.1):
        super().__init__()
        # We will assume the queries, keys, and values all have the same feature size
        self.attn = ScaledDotProductAttention(dropout)
        self.query_tfm = nn.Linear(d_model, d_feature)
        self.key_tfm = nn.Linear(d_model, d_feature)
        self.value_tfm = nn.Linear(d_model, d_feature)

    def forward(self, queries, keys, values, mask=None):
        Q = self.query_tfm(queries) # (Batch, Seq, Feature)
        K = self.key_tfm(keys) # (Batch, Seq, Feature)
        V = self.value_tfm(values) # (Batch, Seq, Feature)
        log_size(Q, "queries, keys, vals")
        # compute multiple attention weighted sums
        x = self.attn(Q, K, V)
        return x

In [16]:
attn_head = AttentionHead(20, 20)
attn_head(q, k, v)

[AttentionHead] queries, keys, vals size=torch.Size([5, 10, 20])


tensor([[[-5.4944e-02,  1.7957e-01, -1.1838e-01, -4.6875e-02, -1.6914e-01,
           1.0049e-01,  2.6786e-01, -2.1204e-01,  1.4735e-01, -1.8410e-01,
           1.0990e-01,  9.9212e-02,  1.5214e-01, -4.1235e-01, -1.3164e-02,
          -4.0524e-01, -1.5269e-01, -1.4364e-01, -3.1397e-01,  4.4045e-01],
         [-5.9720e-02,  1.6309e-01, -1.0697e-01, -5.2231e-02, -1.1765e-01,
           8.0843e-02,  2.5074e-01, -1.8122e-01,  1.5265e-01, -1.7002e-01,
           1.1297e-01,  6.7675e-02,  1.4157e-01, -3.7354e-01, -1.3119e-02,
          -3.4405e-01, -1.4579e-01, -1.3223e-01, -3.0427e-01,  3.9197e-01],
         [-6.8037e-02,  1.5504e-01, -1.1897e-01, -1.8954e-02, -1.4505e-01,
           1.0521e-01,  2.5207e-01, -2.1201e-01,  1.3030e-01, -1.5855e-01,
           1.2356e-01,  6.4783e-02,  1.2035e-01, -3.7260e-01,  3.3556e-03,
          -3.7950e-01, -1.2843e-01, -1.4307e-01, -2.5830e-01,  4.1091e-01],
         [-3.1409e-02,  1.6074e-01, -9.2922e-02, -4.4190e-02, -1.7137e-01,
           1.2676e-01,

The multi-head attention block simply applies multiple attention heads, then concatenates the outputs and applies a single linear projection.

In [17]:
# We'll supress logging from the scaled dot product attention now
logger.setLevel(TensorLoggingLevels.attention_head)

In [18]:
class MultiHeadAttention(nn.Module):
    level = TensorLoggingLevels.multihead_attention_block
    def __init__(self, d_model, d_feature, n_heads, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        self.d_feature = d_feature
        self.n_heads = n_heads
        # in practice, d_model == d_feature * n_heads
        assert d_model == d_feature * n_heads

        # Note that this is very inefficient:
        # I am merely implementing the heads separately because it is 
        # easier to understand this way
        self.attn_heads = nn.ModuleList([
            AttentionHead(d_model, d_feature, dropout) for _ in range(n_heads)
        ])
        self.projection = nn.Linear(d_feature * n_heads, d_model) 
    
    def forward(self, queries, keys, values, mask=None):
        log_size(queries, "Input queries")
        x = [attn(queries, keys, values, mask=mask) # (Batch, Seq, Feature)
             for i, attn in enumerate(self.attn_heads)]
        log_size(x[0], "output of single head")
        
        # reconcatenate
        x = torch.cat(x, dim=Dim.feature) # (Batch, Seq, D_Feature * n_heads)
        log_size(x, "concatenated output")
        x = self.projection(x) # (Batch, Seq, D_Model)
        log_size(x, "projected output")
        return x

In [19]:
heads = MultiHeadAttention(20 * 8, 20, 8)
heads(q.repeat(1, 1, 8), 
      k.repeat(1, 1, 8), 
      v.repeat(1, 1, 8))

[MultiHeadAttention] Input queries size=torch.Size([5, 10, 160])
[AttentionHead] queries, keys, vals size=torch.Size([5, 10, 20])
[AttentionHead] queries, keys, vals size=torch.Size([5, 10, 20])
[AttentionHead] queries, keys, vals size=torch.Size([5, 10, 20])
[AttentionHead] queries, keys, vals size=torch.Size([5, 10, 20])
[AttentionHead] queries, keys, vals size=torch.Size([5, 10, 20])
[AttentionHead] queries, keys, vals size=torch.Size([5, 10, 20])
[AttentionHead] queries, keys, vals size=torch.Size([5, 10, 20])
[AttentionHead] queries, keys, vals size=torch.Size([5, 10, 20])
[MultiHeadAttention] output of single head size=torch.Size([5, 10, 20])
[MultiHeadAttention] concatenated output size=torch.Size([5, 10, 160])
[MultiHeadAttention] projected output size=torch.Size([5, 10, 160])


tensor([[[-0.1361,  0.1967, -0.0143,  ..., -0.0137, -0.0800,  0.3766],
         [-0.1109,  0.1385, -0.1142,  ...,  0.0575, -0.0618,  0.3560],
         [-0.1316,  0.1787, -0.0577,  ..., -0.0099, -0.0858,  0.3920],
         ...,
         [-0.1761,  0.1899, -0.0330,  ...,  0.0197, -0.0750,  0.3744],
         [-0.1167,  0.1407, -0.0622,  ...,  0.0369, -0.0651,  0.4067],
         [-0.1415,  0.1933, -0.0618,  ..., -0.0101, -0.0888,  0.3898]],

        [[-0.2291,  0.1599, -0.0159,  ...,  0.0174, -0.0642,  0.3414],
         [-0.2868,  0.1225, -0.0264,  ...,  0.0438,  0.0078,  0.3615],
         [-0.2686,  0.1321, -0.0515,  ...,  0.0715, -0.0287,  0.3564],
         ...,
         [-0.2327,  0.1424, -0.0644,  ...,  0.0858, -0.0364,  0.3280],
         [-0.2613,  0.0851, -0.0637,  ...,  0.0664, -0.0199,  0.3532],
         [-0.2519,  0.1490, -0.0324,  ...,  0.0291, -0.0154,  0.3227]],

        [[-0.1771,  0.2177, -0.0696,  ...,  0.0302, -0.0724,  0.3467],
         [-0.1663,  0.2216, -0.0125,  ...,  0

### The Encoder

With these core components in place, implementing the encoder is pretty easy.

![image](https://i2.wp.com/mlexplained.com/wp-content/uploads/2017/12/%E3%82%B9%E3%82%AF%E3%83%AA%E3%83%BC%E3%83%B3%E3%82%B7%E3%83%A7%E3%83%83%E3%83%88-2017-12-29-19.14.41.png?w=273)

The encoder consists of the following components:
- A multi-head attention block
- A simple feedforward neural network

These components are connected using residual connections and layer normalization

In [20]:
# We'll supress logging from the individual attention heads
logger.setLevel(TensorLoggingLevels.multihead_attention_block)

Layer normalization is similar to batch normalization, but normalizes across the feature dimension instead of the batch dimension.

![image](https://i1.wp.com/mlexplained.com/wp-content/uploads/2018/01/%E3%82%B9%E3%82%AF%E3%83%AA%E3%83%BC%E3%83%B3%E3%82%B7%E3%83%A7%E3%83%83%E3%83%88-2018-01-11-11.48.12.png?w=1500)

In [21]:
class LayerNorm(nn.Module):
    def __init__(self, d_model, eps=1e-8):
        super(LayerNorm, self).__init__()
        self.gamma = nn.Parameter(torch.ones(d_model))
        self.beta = nn.Parameter(torch.zeros(d_model))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.gamma * (x - mean) / (std + self.eps) + self.beta

The encoder just stacks these together

In [22]:
class EncoderBlock(nn.Module):
    level = TensorLoggingLevels.enc_dec_block
    def __init__(self, d_model=512, d_feature=64,
                 d_ff=2048, n_heads=8, dropout=0.1):
        super().__init__()
        self.attn_head = MultiHeadAttention(d_model, d_feature, n_heads, dropout)
        self.layer_norm1 = LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        self.position_wise_feed_forward = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Linear(d_ff, d_model),
        )
        self.layer_norm2 = LayerNorm(d_model)
        
    def forward(self, x, mask=None):
        log_size(x, "Encoder block input")
        att = self.attn_head(x, x, x, mask=mask)
        log_size(x, "Attention output")
        # Apply normalization and residual connection
        x = x + self.dropout(self.layer_norm1(att))
        # Apply position-wise feedforward network
        pos = self.position_wise_feed_forward(x)
        log_size(x, "Feedforward output")
        # Apply normalization and residual connection
        x = x + self.dropout(self.layer_norm2(pos))
        log_size(x, "Encoder size output")
        return x

In [23]:
enc = EncoderBlock()

In [24]:
enc(torch.rand(5, 10, 512))

[EncoderBlock] Encoder block input size=torch.Size([5, 10, 512])
[MultiHeadAttention] Input queries size=torch.Size([5, 10, 512])
[MultiHeadAttention] output of single head size=torch.Size([5, 10, 64])
[MultiHeadAttention] concatenated output size=torch.Size([5, 10, 512])
[MultiHeadAttention] projected output size=torch.Size([5, 10, 512])
[EncoderBlock] Attention output size=torch.Size([5, 10, 512])
[EncoderBlock] Feedforward output size=torch.Size([5, 10, 512])
[EncoderBlock] Encoder size output size=torch.Size([5, 10, 512])


tensor([[[ 1.9765e+00,  6.0555e-01,  5.7684e-01,  ...,  2.7611e+00,
          -5.1025e-01,  2.7049e+00],
         [ 1.6280e+00,  2.1314e-01,  2.5232e+00,  ...,  1.5289e-01,
          -1.0746e+00,  2.3660e+00],
         [ 1.0024e+00,  9.7015e-01,  1.1129e+00,  ...,  7.8212e-01,
          -1.6309e+00,  3.1498e-02],
         ...,
         [ 5.2415e-01,  4.5764e-01,  9.8830e-01,  ...,  2.7500e+00,
          -5.1298e-01,  7.5724e-01],
         [ 5.0665e-01, -5.2399e-01,  8.6294e-01,  ...,  1.4582e+00,
          -1.0289e+00,  2.0130e+00],
         [ 1.3012e+00,  7.2873e-01,  1.4577e+00,  ...,  8.2640e-01,
          -2.1387e+00,  1.8953e+00]],

        [[ 1.3996e+00,  8.3388e-03,  4.9115e-01,  ...,  2.3502e+00,
          -1.8098e+00,  2.5786e+00],
         [ 1.4870e+00,  5.6864e-01,  1.5955e+00,  ...,  9.2919e-01,
          -1.7983e+00,  2.7972e+00],
         [ 1.6345e+00,  1.2660e-01,  1.0261e+00,  ...,  2.5134e+00,
           5.9002e-01,  1.4710e+00],
         ...,
         [ 1.2663e+00,  4

The encoder consists of 6 consecutive encoder blocks, so can simply be implemented like the following

In [25]:
class TransformerEncoder(nn.Module):
    level = TensorLoggingLevels.enc_dec
    def __init__(self, n_blocks=6, d_model=512,
                 n_heads=8, d_ff=2048, dropout=0.1):
        super().__init__()
        self.encoders = nn.ModuleList([
            EncoderBlock(d_model=d_model, d_feature=d_model // n_heads,
                         d_ff=d_ff, dropout=dropout)
            for _ in range(n_blocks)
        ])
    
    def forward(self, x: torch.FloatTensor, mask=None):
        for encoder in self.encoders:
            x = encoder(x)
        return x

### The Decoder

The decoder is mostly the same as the encoder. There's just one additional multi-head attention block that takes the target sentence as input.

![image](https://i1.wp.com/mlexplained.com/wp-content/uploads/2017/12/%E3%82%B9%E3%82%AF%E3%83%AA%E3%83%BC%E3%83%B3%E3%82%B7%E3%83%A7%E3%83%83%E3%83%88-2017-12-29-19.14.47.png?w=287)

The keys and values are the outputs of the encoder, and the queries are the outputs of the multi-head attention over the target entence embeddings.

In [26]:
class DecoderBlock(nn.Module):
    level = TensorLoggingLevels.enc_dec_block
    def __init__(self, d_model=512, d_feature=64,
                 d_ff=2048, n_heads=8, dropout=0.1):
        super().__init__()
        self.masked_attn_head = MultiHeadAttention(d_model, d_feature, n_heads, dropout)
        self.attn_head = MultiHeadAttention(d_model, d_feature, n_heads, dropout)
        self.position_wise_feed_forward = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Linear(d_ff, d_model),
        )

        self.layer_norm1 = LayerNorm(d_model)
        self.layer_norm2 = LayerNorm(d_model)
        self.layer_norm3 = LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_out, 
                src_mask=None, tgt_mask=None):
        # Apply attention to inputs
        att = self.masked_attn_head(x, x, x, mask=src_mask)
        x = x + self.dropout(self.layer_norm1(att))
        # Apply attention to the encoder outputs and outputs of the previous layer
        att = self.attn_head(queries=x, keys=enc_out, values=enc_out, mask=tgt_mask)
        x = x + self.dropout(self.layer_norm2(att))
        # Apply position-wise feedforward network
        pos = self.position_wise_feed_forward(x)
        x = x + self.dropout(self.layer_norm2(pos))
        return x

In [27]:
dec = DecoderBlock()
dec(torch.rand(5, 10, 512), enc(torch.rand(5, 10, 512)))

[EncoderBlock] Encoder block input size=torch.Size([5, 10, 512])
[MultiHeadAttention] Input queries size=torch.Size([5, 10, 512])
[MultiHeadAttention] output of single head size=torch.Size([5, 10, 64])
[MultiHeadAttention] concatenated output size=torch.Size([5, 10, 512])
[MultiHeadAttention] projected output size=torch.Size([5, 10, 512])
[EncoderBlock] Attention output size=torch.Size([5, 10, 512])
[EncoderBlock] Feedforward output size=torch.Size([5, 10, 512])
[EncoderBlock] Encoder size output size=torch.Size([5, 10, 512])
[MultiHeadAttention] Input queries size=torch.Size([5, 10, 512])
[MultiHeadAttention] output of single head size=torch.Size([5, 10, 64])
[MultiHeadAttention] concatenated output size=torch.Size([5, 10, 512])
[MultiHeadAttention] projected output size=torch.Size([5, 10, 512])
[MultiHeadAttention] Input queries size=torch.Size([5, 10, 512])
[MultiHeadAttention] output of single head size=torch.Size([5, 10, 64])
[MultiHeadAttention] concatenated output size=torch.Siz

tensor([[[-2.6213e+00, -1.5202e+00, -1.5732e+00,  ...,  1.0225e+00,
           1.5174e+00,  7.7989e-01],
         [-2.1316e+00, -1.1355e+00, -1.2760e+00,  ...,  7.8845e-01,
           1.9173e+00,  2.3724e+00],
         [-2.3049e+00, -1.8938e-01, -2.6571e+00,  ...,  3.9799e-01,
           3.4069e+00,  1.1630e+00],
         ...,
         [-2.2333e+00, -2.5128e+00, -1.7321e+00,  ..., -7.5708e-01,
           3.5582e+00,  1.4875e+00],
         [-1.1305e+00, -1.1589e+00, -1.5274e+00,  ...,  1.4933e+00,
           2.9591e+00,  8.0061e-01],
         [-1.2231e+00, -2.4310e+00, -1.9051e+00,  ...,  1.6243e+00,
           3.0201e+00,  2.1192e+00]],

        [[-1.5041e+00, -1.1104e+00, -2.3338e+00,  ...,  1.0043e+00,
           2.7498e+00,  5.9983e-01],
         [-2.2018e+00, -8.9073e-01, -2.0031e+00,  ...,  9.3639e-01,
           2.3343e+00,  1.3269e+00],
         [-2.0905e+00, -1.9472e+00, -1.4471e+00,  ...,  8.9177e-01,
           2.6886e+00,  4.6184e-01],
         ...,
         [-1.8132e+00,  6

Again, the decoder is just a stack of the underlying block so is simple to implement.

In [28]:
class TransformerDecoder(nn.Module):
    level = TensorLoggingLevels.enc_dec
    def __init__(self, n_blocks=6, d_model=512, d_feature=64,
                 d_ff=2048, n_heads=8, dropout=0.1):
        super().__init__()
        self.position_embedding = PositionalEmbedding(d_model)
        self.decoders = nn.ModuleList([
            DecoderBlock(d_model=d_model, d_feature=d_model // n_heads,
                         d_ff=d_ff, dropout=dropout)
            for _ in range(n_blocks)
        ])
        
    def forward(self, x: torch.FloatTensor, 
                enc_out: torch.FloatTensor, 
                src_mask=None, tgt_mask=None):
        for decoder in self.decoders:
            x = decoder(x, enc_out, src_mask=src_mask, tgt_mask=tgt_mask)
        return x

### Positional Embeddings

Attention blocks are just simple matrix multiplications: therefore they don't have any notion of order! The Transformer explicitly adds positional information via the positional embeddings.

In [56]:
class PositionalEmbedding(nn.Module):
    level = 1
    def __init__(self, d_model, max_len=512):
        super().__init__()        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.weight = nn.Parameter(pe, requires_grad=False)
        
    def forward(self, x):
        return self.weight[:, :x.size(1), :] # (1, Seq, Feature)

In [43]:
d_model=160
max_len=512

In [42]:
torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
#torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))

tensor([1.0000e+00, 8.9125e-01, 7.9433e-01, 7.0795e-01, 6.3096e-01, 5.6234e-01,
        5.0119e-01, 4.4668e-01, 3.9811e-01, 3.5481e-01, 3.1623e-01, 2.8184e-01,
        2.5119e-01, 2.2387e-01, 1.9953e-01, 1.7783e-01, 1.5849e-01, 1.4125e-01,
        1.2589e-01, 1.1220e-01, 1.0000e-01, 8.9125e-02, 7.9433e-02, 7.0795e-02,
        6.3096e-02, 5.6234e-02, 5.0119e-02, 4.4668e-02, 3.9811e-02, 3.5481e-02,
        3.1623e-02, 2.8184e-02, 2.5119e-02, 2.2387e-02, 1.9953e-02, 1.7783e-02,
        1.5849e-02, 1.4125e-02, 1.2589e-02, 1.1220e-02, 1.0000e-02, 8.9125e-03,
        7.9433e-03, 7.0795e-03, 6.3096e-03, 5.6234e-03, 5.0119e-03, 4.4668e-03,
        3.9811e-03, 3.5481e-03, 3.1623e-03, 2.8184e-03, 2.5119e-03, 2.2387e-03,
        1.9953e-03, 1.7783e-03, 1.5849e-03, 1.4125e-03, 1.2589e-03, 1.1220e-03,
        1.0000e-03, 8.9125e-04, 7.9433e-04, 7.0795e-04, 6.3096e-04, 5.6234e-04,
        5.0119e-04, 4.4668e-04, 3.9811e-04, 3.5481e-04, 3.1623e-04, 2.8184e-04,
        2.5119e-04, 2.2387e-04, 1.9953e-

In [48]:
torch.zeros(max_len, d_model)[:, 0::2].shape

torch.Size([512, 80])

In [49]:
torch.zeros(max_len, d_model)[:, 1::2].shape

torch.Size([512, 80])

In [55]:
torch.cumsum(input=torch.ones(20), dim=0)[1::2]

tensor([ 2.,  4.,  6.,  8., 10., 12., 14., 16., 18., 20.])

In [36]:
torch.arange(0, 512).unsqueeze(1).float()

tensor([[  0.],
        [  1.],
        [  2.],
        [  3.],
        [  4.],
        [  5.],
        [  6.],
        [  7.],
        [  8.],
        [  9.],
        [ 10.],
        [ 11.],
        [ 12.],
        [ 13.],
        [ 14.],
        [ 15.],
        [ 16.],
        [ 17.],
        [ 18.],
        [ 19.],
        [ 20.],
        [ 21.],
        [ 22.],
        [ 23.],
        [ 24.],
        [ 25.],
        [ 26.],
        [ 27.],
        [ 28.],
        [ 29.],
        [ 30.],
        [ 31.],
        [ 32.],
        [ 33.],
        [ 34.],
        [ 35.],
        [ 36.],
        [ 37.],
        [ 38.],
        [ 39.],
        [ 40.],
        [ 41.],
        [ 42.],
        [ 43.],
        [ 44.],
        [ 45.],
        [ 46.],
        [ 47.],
        [ 48.],
        [ 49.],
        [ 50.],
        [ 51.],
        [ 52.],
        [ 53.],
        [ 54.],
        [ 55.],
        [ 56.],
        [ 57.],
        [ 58.],
        [ 59.],
        [ 60.],
        [ 61.],
        

In [57]:
class WordPositionEmbedding(nn.Module):
    level = 1
    def __init__(self, vocab_size, d_model=512):
        super().__init__()
        self.word_embedding = nn.Embedding(vocab_size, d_model)
        self.position_embedding = PositionalEmbedding(d_model)
        
    def forward(self, x: torch.LongTensor, mask=None) -> torch.FloatTensor:
        return self.word_embedding(x) + self.position_embedding(x)

In [58]:
emb = WordPositionEmbedding(1000)
encoder = TransformerEncoder()

In [59]:
encoder(emb(torch.randint(1000, (5, 30))))

[EncoderBlock] Encoder block input size=torch.Size([5, 30, 512])
[MultiHeadAttention] Input queries size=torch.Size([5, 30, 512])
[MultiHeadAttention] output of single head size=torch.Size([5, 30, 64])
[MultiHeadAttention] concatenated output size=torch.Size([5, 30, 512])
[MultiHeadAttention] projected output size=torch.Size([5, 30, 512])
[EncoderBlock] Attention output size=torch.Size([5, 30, 512])
[EncoderBlock] Feedforward output size=torch.Size([5, 30, 512])
[EncoderBlock] Encoder size output size=torch.Size([5, 30, 512])
[EncoderBlock] Encoder block input size=torch.Size([5, 30, 512])
[MultiHeadAttention] Input queries size=torch.Size([5, 30, 512])
[MultiHeadAttention] output of single head size=torch.Size([5, 30, 64])
[MultiHeadAttention] concatenated output size=torch.Size([5, 30, 512])
[MultiHeadAttention] projected output size=torch.Size([5, 30, 512])
[EncoderBlock] Attention output size=torch.Size([5, 30, 512])
[EncoderBlock] Feedforward output size=torch.Size([5, 30, 512])
[

tensor([[[ 3.1814, -4.3857, -1.5876,  ...,  4.5389,  0.0219,  7.1801],
         [-1.2395, -1.5003, -0.2818,  ...,  2.7318, -0.3822,  4.6980],
         [ 3.1122, -4.5849,  5.4439,  ...,  3.1859, -0.4301,  4.0371],
         ...,
         [ 1.0315, -3.7342,  0.7386,  ...,  4.3112,  0.4267,  4.5148],
         [ 1.7477, -5.4212, -2.6264,  ...,  3.2503,  1.8450,  1.2794],
         [-1.5176, -1.4154,  2.3016,  ...,  6.0029,  0.4483,  3.1732]],

        [[-0.8589, -1.1857, -1.4524,  ..., 13.3574,  3.4316,  2.9086],
         [-2.1845, -3.5757,  0.9847,  ...,  5.1852, -3.8154,  2.8707],
         [ 0.2253, -6.0013,  4.3344,  ...,  5.1273,  1.0284,  3.8588],
         ...,
         [-3.0157, -1.1752,  0.4322,  ...,  2.8387, -0.3303,  3.1197],
         [ 1.0885, -0.5419, -0.1657,  ...,  4.3360,  0.2578,  2.1933],
         [-1.4615, -4.1920,  1.6956,  ...,  6.6987,  2.1902,  3.4976]],

        [[ 0.8237, -2.3076,  1.0450,  ...,  6.6595, -1.7309, -1.5133],
         [ 1.6962, -3.5292, -0.3251,  ...,  3

### Putting it All Together

Let's put everything together now.

![image](https://camo.githubusercontent.com/88e8f36ce61dedfd2491885b8df2f68c4d1f92f5/687474703a2f2f696d6775722e636f6d2f316b72463252362e706e67)

In [60]:
# We'll supress logging from the scaled dot product attention now
logger.setLevel(TensorLoggingLevels.enc_dec_block)

In [61]:
emb = WordPositionEmbedding(1000)
encoder = TransformerEncoder()
decoder = TransformerDecoder()

In [62]:
src_ids = torch.randint(1000, (5, 30))
tgt_ids = torch.randint(1000, (5, 30))
x = encoder(emb(src_ids))
decoder(emb(tgt_ids), x)

[EncoderBlock] Encoder block input size=torch.Size([5, 30, 512])
[EncoderBlock] Attention output size=torch.Size([5, 30, 512])
[EncoderBlock] Feedforward output size=torch.Size([5, 30, 512])
[EncoderBlock] Encoder size output size=torch.Size([5, 30, 512])
[EncoderBlock] Encoder block input size=torch.Size([5, 30, 512])
[EncoderBlock] Attention output size=torch.Size([5, 30, 512])
[EncoderBlock] Feedforward output size=torch.Size([5, 30, 512])
[EncoderBlock] Encoder size output size=torch.Size([5, 30, 512])
[EncoderBlock] Encoder block input size=torch.Size([5, 30, 512])
[EncoderBlock] Attention output size=torch.Size([5, 30, 512])
[EncoderBlock] Feedforward output size=torch.Size([5, 30, 512])
[EncoderBlock] Encoder size output size=torch.Size([5, 30, 512])
[EncoderBlock] Encoder block input size=torch.Size([5, 30, 512])
[EncoderBlock] Attention output size=torch.Size([5, 30, 512])
[EncoderBlock] Feedforward output size=torch.Size([5, 30, 512])
[EncoderBlock] Encoder size output size=t

tensor([[[ 1.1507e+01,  3.6942e+00,  1.1896e+00,  ...,  1.6182e+00,
           3.6384e+00,  2.1935e+00],
         [ 1.0845e+01,  6.0303e+00, -7.7406e-01,  ...,  1.4043e-01,
           3.0258e+00,  8.1023e-01],
         [ 9.9115e+00,  4.8149e+00, -4.1417e+00,  ...,  5.3367e+00,
           1.2551e+00, -3.6881e+00],
         ...,
         [ 7.6631e+00,  6.2149e+00, -9.9306e-01,  ...,  2.7145e+00,
           8.3932e-01, -4.0012e+00],
         [ 1.2607e+01,  5.7130e+00,  3.9997e+00,  ..., -2.6609e+00,
           3.6242e+00, -1.5606e-03],
         [ 8.1767e+00,  8.1239e+00, -4.1463e+00,  ..., -1.4101e+00,
          -1.9761e+00,  3.0817e+00]],

        [[ 8.9693e+00,  1.9953e+00,  1.9182e+00,  ...,  9.4867e-01,
          -8.4640e+00, -8.9125e-01],
         [ 1.0684e+01,  6.0985e+00, -2.4769e+00,  ...,  2.7514e+00,
          -7.4103e+00, -2.5511e+00],
         [ 7.2011e+00,  6.4578e-01,  5.0101e+00,  ...,  4.2513e+00,
          -5.8103e+00, -2.3295e+00],
         ...,
         [ 9.2791e+00,  3